# This code implements the training pipeline for various models

In [8]:
import torch
import torch.nn as nn
from torch.optim import Adam
from dataformatter import *
from models import *
import matplotlib.pyplot as plt
import os

In [9]:
# Listing all the hyper-parameters that need to be set here
BATCH_SZ = 32
MODEL_TYPE = 'FC' # OPTIONS ARE : [FC, TRANSFORMER, LSTM]
LR = 1e-3
DATA_PATH = 'hawkeye_trace_belady_graph.csv' # This is the CSV FILE WE ARE TRYING TO ANALYZE
SAVE_FLDR = 'results'
N_EPOCHS = 20

In [10]:
def run_epoch(model, optimizer, data_iterator, mode='train'):
    model.train()
    if mode == 'val':
        model.eval()
    stats = []
    for batch in data_iterator:
        # we get the loss from passing the batch to the model
        # each model will have it's own way of deadling with the data [we can jointly figure this out]
        loss, acc = model(batch)
        stats.append([loss.item(), acc.item()])
        if mode == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return np.array(stats)

In [11]:
def graph_results(data, desc):
    plt.title(desc)
    plt.plot(range(len(data)), data[:, 0], color='tab:red', label='Loss')
    plt.plot(range(len(data)), data[:, 0], color='tab:blue', label='Accuracy')
    plt.legend()
    plt.savefig("{}/{}.png".format(SAVE_FLDR, desc.replace(' ', '_')))
    plt.show()

In [12]:
def trainer(model, optimizer, dataset, num_epochs=20, desc='Description of model'):
    # Todo [all]
    # Figure out how to split the data into a train-val-test regime
    stats = []
    for epoch_ in range(num_epochs):
        # get a data iterator for this epoch
        data_iter = get_batch_iterator(data, BATCH_SZ)
        epoch_stats = run_epoch(model, optimizer, data_iter, mode='train')
        # Todo[all] would normally run the validation set at this point
        stats.append(epoch_stats.mean(axis=0))
    graph_results(stats, desc)
    return model

In [13]:
if not os.path.exists(SAVE_FLDR):
    os.makedirs(SAVE_FLDR)

model = get_model(MODEL_TYPE)
optimizer = Adam(model.parameters(), lr=LR)
dataset = csv_to_data(DATA_PATH)
model.prep_for_data(dataset)

NameError: name 'FC_CONFIG' is not defined

In [ ]:
model = trainer(model, optimizer, dataset, N_EPOCHS)